In [ ]:
#Avec lmfit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lmfit
import os


fichier = "..." #Ecrire le fichier sous la forme "Sample_ODMR_data.dat"
df = pd.read_csv(fichier, sep="\t")

basename = os.path.splitext(os.path.basename(fichier))[0]
bname = basename.split("_")
Sample = bname[0]

x_data = df['frequency (Hz)'].values
y_data = df['count data (counts/s)'].values


mask = (x_data >= 3.5e9) & (x_data <= 3.9e9)
x_data = x_data[mask]
y_data = y_data[mask]
R=y_data[0]
ydataN=y_data/np.max(y_data)
contrast = np.min(y_data)-np.max(y_data)
CO=1-np.min(ydataN)

#LARGUEUR MI HAUTEUR
def FWHM(sigma):
  return 2*np.sqrt(2*np.log(2))*sigma

#A=amplitude, mu= position centrale des pics, sigma = largeur du pic
def gaussienne(x, A, mu, sigma):
    return A * np.exp(-((x - mu)**2) / (2 * sigma**2))

def four_gauss(params, x, y):
    model = (gaussienne(x, params['A1'], params['mu']-params['shift'], params['sigma'])+
             gaussienne(x, params['A2'], params['mu'], params['sigma'])+
             gaussienne(x, params['A3'], params['mu']+params['shift'], params['sigma'])+
             gaussienne(x, params['A4'], params['mu']+2*params['shift'], params['sigma'])+
             params['offset'])
    return model - y

# Initialisation des paramètres (initial_guess réalistes)
params = lmfit.Parameters()

params.add('shift', value=67e6, vary = True, min =40e6, max =80e6)
params.add('A1', value=contrast/3, vary = True, min =0.8*contrast/3, max =1.2*contrast/3)
params.add('A4', value=contrast/3, vary = True, min =0.8*contrast/3, max =1.2*contrast/3)
params.add('A2', value=contrast, vary = True, min =0.8*contrast, max =1.2*contrast)
params.add('A3', value=contrast, vary = True, min =0.8*contrast, max =1.2*contrast)
params.add('mu', value=3.65e9, vary = True, min =3.6e9, max =3.7e9)
params.add('sigma', value=10e6, vary = True, min =1e6, max =50e6)
params.add('offset', value=np.max(y_data), vary = True)


# Fit avec lmfit
plt.figure()
result = lmfit.minimize(four_gauss, params, args = (x_data, y_data))

x_fit = np.linspace(x_data[0], x_data[-1], 1000)
fit = four_gauss(result.params, x_fit, np.zeros((len(x_fit))))

plt.plot(x_data, y_data, marker ='o', linestyle ='')
plt.plot(x_fit, fit, color ='red', linestyle ='-',label='fit')

plt.xlabel("Fréquence (GHz)")
plt.ylabel("PL (coups/s)")
plt.legend()
plt.title("")
plt.grid()



C_percent = CO * 100
fwhm = FWHM(result.params["sigma"].value)



#lmfit.report_fit(result)
def sensibility(R,CO,fwhm):
  gamma_e = 28e9
  facteur=0.7
  return facteur*(1/(2.8e10))*(fwhm/(CO*np.sqrt(R)))*1e6

output_file = f"Variable_ODMR.txt"
if os.path.exists(output_file):
  mode = 'a' #Ajout
else:
  mode = 'w' #Creation

with open(output_file,mode) as file:
  if mode == 'w':
    file.write("Sample\tR\tC\tC_percent\tfwhm\tSensibility( µT / √Hz)\n") #sensibilité en µT / √Hz
  file.write(f"{Sample}\t{R:.4f}\t{CO:.4f}\t{C_percent:.4f} (%)\t{fwhm:.4f}\t{sensibility(R,CO,fwhm)}\n")


graphique_sortie = f"{Sample}_ODMR_fit.png"

plt.savefig(graphique_sortie, dpi=300)
plt.show()

print(f"Graphique sauvegardé sous : {graphique_sortie}")
print(f"Les variables ont été sauvegardés dans '{output_file}'")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.5 MB/s eta 0:00:00


FileNotFoundError: [Errno 2] No such file or directory: '10AEIRIS_ODMR_data.dat'